In [2]:
import pandas as pd

df = pd.read_csv('../../data/train.csv')
concerts = pd.read_csv('../../data/concerts.csv')
planned_concerts = pd.read_csv('../../data/concerts_2014-15.csv')
subscriptions = pd.read_csv('../../data/subscriptions.csv')

In [3]:
subs_by_acc = subscriptions.groupby(['account.id'])

subscriptions_by_account = pd.DataFrame({'num_subscriptions':subs_by_acc.size(), 
                                        'sub_tier': subs_by_acc['subscription_tier'].apply(lambda x: x.mode().iloc[0]),
                                        'sub_seasons': subs_by_acc['season'].unique()
                                        }).reset_index()

# group concerts by season and aggregate list of unique conductors
concerts['conductor'] = concerts['who'].apply(lambda x: x.split(',')[0])
conductors_by_season = concerts.groupby(['season'])['conductor'].unique().reset_index()

# create new column conductors in subscriptions_by_account with all the unique values as a flattened list from conductors_by_season where the season is one of the sub_seasons
subscriptions_by_account['conductors'] = subscriptions_by_account['sub_seasons'].apply(lambda x: set([item for sublist in conductors_by_season[conductors_by_season['season'].isin(x)]['conductor'] for item in sublist]))

# transform "who" column in planned_concerts to "conductors" column with just the name of the conductors
planned_concerts['conductors'] = planned_concerts['who'].apply(lambda x: x.split(',')[0])
#display(planned_concerts.head())
# aggregate list of unique conductors in next season
planned_conductors = planned_concerts['conductors'].unique()
subscriptions_by_account['watched_conductors'] = subscriptions_by_account['conductors'].apply(lambda x: len(x.intersection(planned_conductors)))
#display(planned_conductors)
#display(conductors_by_season)           
#display(subscriptions_by_account.iloc[0]['conductors'])
#display(subscriptions_by_account['watched_conductors'].value_counts())
#display(subscriptions_by_account[subscriptions_by_account['conductors'].apply(lambda x: len(x)) == 8])
subscriptions_by_account.drop(['sub_seasons', 'conductors'], axis=1, inplace=True)


df['watched_conductors'] = df['account.id'].map(subscriptions_by_account.set_index('account.id')['watched_conductors'])
df['watched_conductors'] = df['watched_conductors'].fillna(0)

# plot the relation between watched conductors and label
display(df.groupby(['label'])['watched_conductors'].mean())

# measure correlation between watched conductors and label
display(df[['label', 'watched_conductors']].corr())

label
0    0.051899
1    1.704819
Name: watched_conductors, dtype: float64

,label,watched_conductors
label,1.000000,0.749874
watched_conductors,0.749874,1.000000
